# Path of Exile game stats data eda

dates: 2022.04.25 ~ 

This file contains an analysis processing of the game 'Path of Exile' data.

###### Origins of datas from here: 
poe_stats : https://www.kaggle.com/datasets/gagazet/path-of-exile-league-statistic

Data contains stats of 59000 players, from 4th August of 2017 and before now.

---

This file made for EDA after pre-processing the original data, arbitrarily setted by me.

In this file, the rank column was considered problemed so I revised little.

# Dataset and libraries 

## data and libraries import

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from scipy.stats import trim_mean
from statsmodels import robust
import wquantiles

df = pd.read_csv('poe_stats_1preprocessing.csv')

In [2]:
df

,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
0,1,False,False,Tzn_NecroIsFineNow,100,Necromancer,3dcddd59f5088893f734f39686350990dae168cc4f4b00...,4250334444,TheTzn,40,thetzn,Harbinger,1,0,1,0,1
1,1,False,False,RaizNeverFirstQT,100,Necromancer,8f3216db5ac9106c287a834731aafc83c387138f28fc45...,4250334444,RaizQT,22,raizqt,SSF Harbinger HC,1,0,1,0,1
2,1,False,False,GucciStreamerAdvantage,100,Necromancer,c6ec2dae3855c551e0597c06ef2da06fbb5512487ded7f...,4250334444,GucciPradas,35,guccipradas,Hardcore Harbinger,1,0,1,0,1
3,1,False,False,ChiroxPrime,100,Slayer,c861372da792be0b22c45bf437ccd58437c52e9455e0bf...,4250334444,Chiroxun,26,NaN,SSF Harbinger,1,1,0,1,2
4,2,False,False,Cool_NecroIsFineNow,100,Deadeye,24ae924ceed7989ef3d3d6772612832bb467a6094352c7...,4250334444,cooltail,22,NaN,Harbinger,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59771,14999,True,False,ПроклятьеРекласта,89,Necromancer,d33b4f6e08c10e365765f9a36a8f36d561fd1d86f10ee1...,1786695653,Romanchik002,18,romanchik002,Hardcore Harbinger,1,0,1,0,1
59772,15000,False,False,IshibashiSummoner,94,Necromancer,5764cfa387e0a87a4bebc1a3c5017e92de8bbb06445c47...,2720770853,Ishibashisan,33,NaN,Harbinger,1,0,1,0,1
59773,15000,False,False,BLively,73,Slayer,9ac75ab75a47cee8a9dfb0a31912df8909720a8b20f9f1...,495034029,ruby01,19,NaN,SSF Harbinger,1,1,0,1,2
59774,15000,False,False,vawddvaw,89,Gladiator,cf02dfc0c90b2df9c7ac76bbedd91e93c2a8a2ca629c15...,1786638972,Lecarn,24,lecarn,Hardcore Harbinger,1,1,0,1,2


In [3]:
df.shape

(59776, 17)

In [4]:
df.sample(10)

,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
22684,5685,False,False,zspartacusz,96,Gladiator,f7ea8706d99590291f307360953aee320328dfd37d2e16...,3111806880,zTyrerz,30,tyr3r,Harbinger,1,1,0,1,2
39133,9813,True,False,ExtremlyFluffyHamster,67,Necromancer,7a02000e8dd18b164acbbc1db1b7a8b5cc0d763e1ca831...,287450075,teodosie1,14,NaN,SSF Harbinger HC,1,0,1,0,1
3795,951,False,False,PekaHC,89,Gladiator,ecf43630fcf4321987a2e2ca74c71004f8d2cf6bfc7326...,1825703575,peka12345,18,NaN,SSF Harbinger HC,1,1,0,1,2
44052,11051,False,False,Ranjor,90,Raider,82e5f1d320ec9eeb651418e6b37c9a632dd961a2897926...,1947990318,khamme,24,NaN,Hardcore Harbinger,1,0,0,1,1
8782,2201,True,False,NugiyensHottestFangirl,85,Necromancer,3b32333fc9a407dc1f072925ab5a9b7935f984483d1b7f...,1325838529,FixiInArsh,13,NaN,SSF Harbinger HC,1,0,1,0,1
17603,4412,True,False,Amashiki_L,92,Slayer,b93aae521a4750635b0d70fa482a4fdb5e815e644d1781...,2381684585,seren8,26,NaN,Hardcore Harbinger,1,1,0,1,2
33468,8390,True,False,Molotov_Koktel,69,Inquisitor,c873fb8aea1b3441b9862dc085180a8a41687508a8ceb3...,361895440,WaRz1Kill,21,NaN,SSF Harbinger HC,1,1,1,0,2
38791,9727,False,False,Mf_kinetic,95,Pathfinder,15745891eb659697b4a9f5c8eef96f90aeef6d756caadb...,2881779396,Oleg_Smel,40,NaN,Harbinger,1,0,0,1,1
56612,14206,False,False,FurryTotPact,74,Hierophant,1aa7ab4d0becaeb321425cb1c0a4de7851c7bfe5a16a94...,530881981,madox2069,26,NaN,SSF Harbinger,1,1,1,0,2
46106,11567,False,False,QWERTTTTTTTTTTTTTTTTTTT,95,Pathfinder,66065ba2a1d7a83204432ae96ded662f09cfe60fda3988...,2876715938,QWERT170893,29,NaN,Harbinger,1,0,0,1,1


## data describes

In [5]:
# columns name

df.columns

Index(['rank', 'dead', 'online', 'name', 'level', 'class', 'id', 'experience',
       'account', 'challenges', 'twitch', 'ladder', 'special_class',
       'stat_Str', 'stat_Int', 'stat_Dex', 'stats_num'],
      dtype='object')

In [6]:
df.describe()

,rank,level,experience,challenges,special_class,stat_Str,stat_Int,stat_Dex,stats_num
count,59776.000000,59776.000000,5.977600e+04,59776.000000,59776.000000,59776.000000,59776.000000,59776.000000,59776.000000
mean,7495.369379,85.422578,1.825786e+09,25.759569,0.991702,0.582207,0.418027,0.513450,1.513684
std,4330.179898,11.288017,1.094748e+09,8.440888,0.090713,0.493200,0.493239,0.499823,0.614134
min,1.000000,53.000000,7.790982e+07,4.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,3745.000000,78.000000,7.774697e+08,19.000000,1.000000,0.000000,0.000000,0.000000,1.000000
50%,7492.000000,90.000000,1.934010e+09,25.000000,1.000000,1.000000,0.000000,1.000000,1.000000
75%,11247.000000,95.000000,2.876117e+09,33.000000,1.000000,1.000000,1.000000,1.000000,2.000000
max,15000.000000,100.000000,4.250334e+09,40.000000,1.000000,1.000000,1.000000,1.000000,3.000000


In [7]:
df.describe(include='object')

,name,class,id,account,twitch,ladder
count,59776,59776,59776,59776,16300,59776
unique,58286,26,59776,37123,9613,4
top,_Flic_,Berserker,3dcddd59f5088893f734f39686350990dae168cc4f4b00...,chuz101,thespunkylobster,SSF Harbinger
freq,14,8711,1,30,18,14981


In [8]:
df.nunique()

rank             15000
dead                 2
online               2
name             58286
level               48
class               26
id               59776
experience       54124
account          37123
challenges          37
twitch            9613
ladder               4
special_class        2
stat_Str             2
stat_Int             2
stat_Dex             2
stats_num            3
dtype: int64

In [9]:
df.isnull().sum()

rank                 0
dead                 0
online               0
name                 0
level                0
class                0
id                   0
experience           0
account              0
challenges           0
twitch           43476
ladder               0
special_class        0
stat_Str             0
stat_Int             0
stat_Dex             0
stats_num            0
dtype: int64

### describe results

데이터 총 개수 59776 개

컬럼명: ['rank', 'dead', 'online', 'name', 'level', 'class', 'id', 'experience', 'account', 'challenges', 'twitch', 'ladder']

랭크는 총 1만 5천개, 죽음과 온라인 여부는 True, False로 이루어져있다.

id가 오직 unique 하며, 한 아이디당 여러 account를 가질 수 있는 것으로 보인다. 그에 따라 일부 name은 겹치는 것을 보인다.

level 과 class는 게임 내 레벨과 직업으로 이루어져 있다.

experience 는 게임 내 경험치를 의미하는 것일 것이며,

twitch는 트위치 계정이 적혀있는 컬럼으로 보인다. 일부 살펴본 결과 한 트위치 아이디로 여러 id를 운영하며 스트리밍 하는 것으로 보인다. 최대 18 아이디를 운영하는 twitch id도 있다.

challenges 의 경우 poe 게임이 던전형 rpg임을 감안하여 짐작해보건대 해당 랭크를 따기까지 해당 모드를 몇 번 플레이 했는지를 나타내는 것으로 예상된다.

ladder 의 경우 플레이모드로, SSF는 솔로 모드를 뜻하고 Hardcore = HC 로 추정된다.


### in addition

*분석의 목적을 정확히 정하고 싶지만 이 게임에서 랭킹에 드는 기준조차 제대로 모르기 때문에, 랭커들의 직업이나 특성들을 찾아보는 것을 주 목적으로 하되, 데이터로 게임(맥락)을 이해할 수 있을지 궁금한 부분도 있어서 최대한 많은 생각을 하면서 분석을 진행하려 한다.*

## EDA planning

이에 따라 현 데이터 셋은 크게 이렇게 나누어 볼 수 있을 것이다.

1. rank (rank bins) 
2. level or exp bins
3. class or stats
4. ladder

또한 위의 내용을 laddar 로 나누어 다시 나누어 볼 수 있을 것이다.

## But first, I think we should look by ladder (game mode)

1. 게임 모드별로 나눈다.

In [10]:
df['ladder'].describe()

count             59776
unique                4
top       SSF Harbinger
freq              14981
Name: ladder, dtype: object

In [11]:
df['rank'].value_counts()

1        4
9950     4
9938     4
9939     4
9940     4
        ..
13443    3
3314     3
1506     3
6477     3
2367     2
Name: rank, Length: 15000, dtype: int64

In [12]:
df['rank'].value_counts().max()

4

각 랭크는 최대 4명으로 되어있다. 이것만 보아도 게임 모드에 따라 랭크가 구분되어 있음을 미루어 짐작해 볼 수 있다. 확인해본다.

In [13]:
pd.pivot_table(data = df, index = ['ladder'], aggfunc = 'count')[['account', 'rank']]

,account,rank
ladder,,
Harbinger,14918,14918
Hardcore Harbinger,14905,14905
SSF Harbinger,14981,14981
SSF Harbinger HC,14972,14972


아이디의 개수와 랭크의 개수가 일치하는 것으로 나왔다. 게임 모드에 따라 랭크가 구분되어 있음이 맞다.

개인적인 생각으로, 이 데이터는 rank별로 나열될 것이 아니라, ladder 별로 rank가 나열되어야 하는 것이 맞다고 생각이 된다.



In [14]:
# 각 모드의 id, rank 고유 개수

modes = ['Harbinger', 'Hardcore Harbinger', 'SSF Harbinger', 'SSF Harbinger HC']

for i in modes:
    print("ladder: ", i)
    print("values of unique id: ", df[df['ladder'] == i]['id'].nunique())
    print("values of unique rank: ", df[df['ladder'] == i]['rank'].nunique())
    print("max values of rank: ", df[df['ladder'] == i]['rank'].max())
    print('-------------------------------')

ladder:  Harbinger
values of unique id:  14918
values of unique rank:  14918
max values of rank:  15000
-------------------------------
ladder:  Hardcore Harbinger
values of unique id:  14905
values of unique rank:  14905
max values of rank:  15000
-------------------------------
ladder:  SSF Harbinger
values of unique id:  14981
values of unique rank:  14981
max values of rank:  15000
-------------------------------
ladder:  SSF Harbinger HC
values of unique id:  14972
values of unique rank:  14972
max values of rank:  15000
-------------------------------


In [15]:
df[df['ladder'] == 'Harbinger']['rank']

0            1
4            2
8            3
13           4
18           5
         ...  
59757    14996
59761    14997
59766    14998
59770    14999
59772    15000
Name: rank, Length: 14918, dtype: int64

분명 모든 게임 모드에서 15000 등까지 있는 것으로 나타나지만 nunique를 넣어보면 일부 사라져 있다.

랭킹 개수 < 최대 랭크넘버 이기 때문에 동점자가 있는 것 같지는 않고, 크롤링 상의 문제이거나, 내부 데이터 상의 어떤 그런 것에 의해서 일부 없는 것으로 생각된다. (게임모드에서는 결측치가 없었기 때문)

등수는 절대적인 수치가 아니고, 나열 되어서 순서가 있을 때 의미가 존재하기 때문에 없는 등수를 채워서 총 14918 등까지 나타나도록 수정할 것이다.

## df divising

In [16]:
df_Har = df[df['ladder'] == 'Harbinger']
df_HarHC = df[df['ladder'] == 'Hardcore Harbinger']
df_SHar = df[df['ladder'] == 'SSF Harbinger']
df_SHarHC = df[df['ladder'] == 'SSF Harbinger HC']

In [17]:
df_Har

,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
0,1,False,False,Tzn_NecroIsFineNow,100,Necromancer,3dcddd59f5088893f734f39686350990dae168cc4f4b00...,4250334444,TheTzn,40,thetzn,Harbinger,1,0,1,0,1
4,2,False,False,Cool_NecroIsFineNow,100,Deadeye,24ae924ceed7989ef3d3d6772612832bb467a6094352c7...,4250334444,cooltail,22,NaN,Harbinger,1,0,0,1,1
8,3,False,False,Rami_The_Negromaniac,100,Necromancer,00c0030fc548cc4c0d804564f8fb356610044b7e6d484f...,4250334444,rami1337,40,rami1337,Harbinger,1,0,1,0,1
13,4,False,False,Guraj_NecroIsFineNow,100,Raider,34bedf0408f53151dcc31eb43c3bd10a9fb0d7ba32b95b...,4250334444,Guraj93,24,guraj_hots,Harbinger,1,0,0,1,1
18,5,False,False,Ryphe,100,Occultist,b6387a6be18939980798db2ec8e8a2924027fa803e063a...,4250334444,Ryphe,40,rypherious,Harbinger,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59757,14996,False,True,iTrofiiforTi,94,Ascendant,575bcf4246a6287f2cd19f3749bb9e803cd59b1f378cc4...,2720826701,troffi,40,NaN,Harbinger,1,1,1,1,3
59761,14997,False,False,Semirf,94,Chieftain,88f31e056e4ea4c56c1fb5a4a6b5e02ba1ec093ffa028c...,2720815490,priima,26,semikreisi,Harbinger,1,1,0,0,1
59766,14998,False,False,Vicoachh,94,Raider,8f55b48a26bfaf96bb0f5cebf6af76b27ad2eedd6dbdd6...,2720813347,ViCoach,28,NaN,Harbinger,1,0,0,1,1
59770,14999,False,False,Imastunboy,94,Berserker,5f6c5f759dc46edfcfd6292c792a8dcd33bada2bf070b4...,2720777728,Gangbash,22,NaN,Harbinger,1,1,0,0,1


In [18]:
df_Har['rank'].nunique()

14918

rank 는 15000 까지 있지만 row는 12918 로 일부 유실되어있다. 이는 df_Har 뿐만이 아닐 것이다.

어떠한 이유에서든 없는 것은 지금 다시 가져올 수 없으므로, 빠지는 값이 없이 해주는 것이 더 정확한 계산을 가져올 수 있을 것이라 생각한다.

### Insert missing values in 'rank'

구분해주기 위해 기존의 rank를 rank를 덮는다.

In [19]:
# 기존 rank에 rank를 다시 매긴다. ascending = True : 작은 순대로 오름차순 1부터 순위 매김.

df_Har['rank'] = df_Har['rank'].rank(ascending = True)
df_Har

/var/folders/qw/j72d9nwn5dq0vxtqtw2bbtnh0000gp/T/ipykernel_2391/392104618.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Har['rank'] = df_Har['rank'].rank(ascending = True)


,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
0,1.0,False,False,Tzn_NecroIsFineNow,100,Necromancer,3dcddd59f5088893f734f39686350990dae168cc4f4b00...,4250334444,TheTzn,40,thetzn,Harbinger,1,0,1,0,1
4,2.0,False,False,Cool_NecroIsFineNow,100,Deadeye,24ae924ceed7989ef3d3d6772612832bb467a6094352c7...,4250334444,cooltail,22,NaN,Harbinger,1,0,0,1,1
8,3.0,False,False,Rami_The_Negromaniac,100,Necromancer,00c0030fc548cc4c0d804564f8fb356610044b7e6d484f...,4250334444,rami1337,40,rami1337,Harbinger,1,0,1,0,1
13,4.0,False,False,Guraj_NecroIsFineNow,100,Raider,34bedf0408f53151dcc31eb43c3bd10a9fb0d7ba32b95b...,4250334444,Guraj93,24,guraj_hots,Harbinger,1,0,0,1,1
18,5.0,False,False,Ryphe,100,Occultist,b6387a6be18939980798db2ec8e8a2924027fa803e063a...,4250334444,Ryphe,40,rypherious,Harbinger,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59757,14914.0,False,True,iTrofiiforTi,94,Ascendant,575bcf4246a6287f2cd19f3749bb9e803cd59b1f378cc4...,2720826701,troffi,40,NaN,Harbinger,1,1,1,1,3
59761,14915.0,False,False,Semirf,94,Chieftain,88f31e056e4ea4c56c1fb5a4a6b5e02ba1ec093ffa028c...,2720815490,priima,26,semikreisi,Harbinger,1,1,0,0,1
59766,14916.0,False,False,Vicoachh,94,Raider,8f55b48a26bfaf96bb0f5cebf6af76b27ad2eedd6dbdd6...,2720813347,ViCoach,28,NaN,Harbinger,1,0,0,1,1
59770,14917.0,False,False,Imastunboy,94,Berserker,5f6c5f759dc46edfcfd6292c792a8dcd33bada2bf070b4...,2720777728,Gangbash,22,NaN,Harbinger,1,1,0,0,1


In [20]:
# 정수 타입으로 바꾸어준다.

df_Har['rank'] = df_Har['rank'].astype('int64', copy = True)

/var/folders/qw/j72d9nwn5dq0vxtqtw2bbtnh0000gp/T/ipykernel_2391/766561525.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Har['rank'] = df_Har['rank'].astype('int64', copy = True)


In [21]:
# checking

df_Har

,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
0,1,False,False,Tzn_NecroIsFineNow,100,Necromancer,3dcddd59f5088893f734f39686350990dae168cc4f4b00...,4250334444,TheTzn,40,thetzn,Harbinger,1,0,1,0,1
4,2,False,False,Cool_NecroIsFineNow,100,Deadeye,24ae924ceed7989ef3d3d6772612832bb467a6094352c7...,4250334444,cooltail,22,NaN,Harbinger,1,0,0,1,1
8,3,False,False,Rami_The_Negromaniac,100,Necromancer,00c0030fc548cc4c0d804564f8fb356610044b7e6d484f...,4250334444,rami1337,40,rami1337,Harbinger,1,0,1,0,1
13,4,False,False,Guraj_NecroIsFineNow,100,Raider,34bedf0408f53151dcc31eb43c3bd10a9fb0d7ba32b95b...,4250334444,Guraj93,24,guraj_hots,Harbinger,1,0,0,1,1
18,5,False,False,Ryphe,100,Occultist,b6387a6be18939980798db2ec8e8a2924027fa803e063a...,4250334444,Ryphe,40,rypherious,Harbinger,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59757,14914,False,True,iTrofiiforTi,94,Ascendant,575bcf4246a6287f2cd19f3749bb9e803cd59b1f378cc4...,2720826701,troffi,40,NaN,Harbinger,1,1,1,1,3
59761,14915,False,False,Semirf,94,Chieftain,88f31e056e4ea4c56c1fb5a4a6b5e02ba1ec093ffa028c...,2720815490,priima,26,semikreisi,Harbinger,1,1,0,0,1
59766,14916,False,False,Vicoachh,94,Raider,8f55b48a26bfaf96bb0f5cebf6af76b27ad2eedd6dbdd6...,2720813347,ViCoach,28,NaN,Harbinger,1,0,0,1,1
59770,14917,False,False,Imastunboy,94,Berserker,5f6c5f759dc46edfcfd6292c792a8dcd33bada2bf070b4...,2720777728,Gangbash,22,NaN,Harbinger,1,1,0,0,1


### also, remained 'ladders'

##### df_HarHC

##### df_HarHC

In [22]:
# 기존 rank에 rank를 다시 매긴다. ascending = True : 작은 순대로 오름차순 1부터 순위 매김.

df_HarHC['rank'] = df_HarHC['rank'].rank(ascending = True)
df_HarHC

/var/folders/qw/j72d9nwn5dq0vxtqtw2bbtnh0000gp/T/ipykernel_2391/687538335.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_HarHC['rank'] = df_HarHC['rank'].rank(ascending = True)


,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
2,1.0,False,False,GucciStreamerAdvantage,100,Necromancer,c6ec2dae3855c551e0597c06ef2da06fbb5512487ded7f...,4250334444,GucciPradas,35,guccipradas,Hardcore Harbinger,1,0,1,0,1
5,2.0,False,False,SteelStreamerAdvantage,100,Deadeye,51acb8e6fac3cfaa7ff848151dad01b0a6f350e7525543...,4250334444,Steelmage,37,steelmage202,Hardcore Harbinger,1,0,0,1,1
10,3.0,False,False,YovuStreamerAdvantage,100,Champion,9956e628b68887b2c33f430d1cff3736da0bb6e2aac2b6...,4250334444,Yovu,32,yovu,Hardcore Harbinger,1,1,0,1,2
12,4.0,False,False,Havoc_KAPOW,100,Inquisitor,106abe85e0bda651fdab9b265eff93b4bf47d78d5ebf81...,4250334444,Havoc6,25,havoc616,Hardcore Harbinger,1,1,1,0,2
16,5.0,False,False,TWITCHTV_TARKECAT,100,Slayer,30208acbf537304cf18efe02dc624f55278acd99e4ee45...,4250334444,Kanelol,31,kanelife,Hardcore Harbinger,1,1,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59758,14901.0,True,False,DontPutYourDickInTheFan,89,Inquisitor,83301ecf4bd1c3af31ec8e08af92cbb87df79d442162f1...,1786817480,Nordic87,20,nordic_87,Hardcore Harbinger,1,1,1,0,2
59763,14902.0,True,False,Miranish,89,Slayer,fc7e1403a00e0305bf42be88bbdcceb1250e28a96cd933...,1786735583,le0nkun,35,NaN,Hardcore Harbinger,1,1,0,1,2
59767,14903.0,False,False,gewan,89,Gladiator,d325ae51d8c9ac1c5b32d0d83de483c744d12630c82b50...,1786730286,gewan,22,NaN,Hardcore Harbinger,1,1,0,1,2
59771,14904.0,True,False,ПроклятьеРекласта,89,Necromancer,d33b4f6e08c10e365765f9a36a8f36d561fd1d86f10ee1...,1786695653,Romanchik002,18,romanchik002,Hardcore Harbinger,1,0,1,0,1


In [23]:
# 정수 타입으로 바꾸어준다.

df_HarHC['rank'] = df_HarHC['rank'].astype('int64', copy = True)

/var/folders/qw/j72d9nwn5dq0vxtqtw2bbtnh0000gp/T/ipykernel_2391/3704664788.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_HarHC['rank'] = df_HarHC['rank'].astype('int64', copy = True)


In [24]:
# checking

df_HarHC

,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
2,1,False,False,GucciStreamerAdvantage,100,Necromancer,c6ec2dae3855c551e0597c06ef2da06fbb5512487ded7f...,4250334444,GucciPradas,35,guccipradas,Hardcore Harbinger,1,0,1,0,1
5,2,False,False,SteelStreamerAdvantage,100,Deadeye,51acb8e6fac3cfaa7ff848151dad01b0a6f350e7525543...,4250334444,Steelmage,37,steelmage202,Hardcore Harbinger,1,0,0,1,1
10,3,False,False,YovuStreamerAdvantage,100,Champion,9956e628b68887b2c33f430d1cff3736da0bb6e2aac2b6...,4250334444,Yovu,32,yovu,Hardcore Harbinger,1,1,0,1,2
12,4,False,False,Havoc_KAPOW,100,Inquisitor,106abe85e0bda651fdab9b265eff93b4bf47d78d5ebf81...,4250334444,Havoc6,25,havoc616,Hardcore Harbinger,1,1,1,0,2
16,5,False,False,TWITCHTV_TARKECAT,100,Slayer,30208acbf537304cf18efe02dc624f55278acd99e4ee45...,4250334444,Kanelol,31,kanelife,Hardcore Harbinger,1,1,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59758,14901,True,False,DontPutYourDickInTheFan,89,Inquisitor,83301ecf4bd1c3af31ec8e08af92cbb87df79d442162f1...,1786817480,Nordic87,20,nordic_87,Hardcore Harbinger,1,1,1,0,2
59763,14902,True,False,Miranish,89,Slayer,fc7e1403a00e0305bf42be88bbdcceb1250e28a96cd933...,1786735583,le0nkun,35,NaN,Hardcore Harbinger,1,1,0,1,2
59767,14903,False,False,gewan,89,Gladiator,d325ae51d8c9ac1c5b32d0d83de483c744d12630c82b50...,1786730286,gewan,22,NaN,Hardcore Harbinger,1,1,0,1,2
59771,14904,True,False,ПроклятьеРекласта,89,Necromancer,d33b4f6e08c10e365765f9a36a8f36d561fd1d86f10ee1...,1786695653,Romanchik002,18,romanchik002,Hardcore Harbinger,1,0,1,0,1


##### df_SHar

In [25]:
# 기존 rank에 rank를 다시 매긴다. ascending = True : 작은 순대로 오름차순 1부터 순위 매김.

df_SHar['rank'] = df_SHar['rank'].rank(ascending = True)
df_SHar

/var/folders/qw/j72d9nwn5dq0vxtqtw2bbtnh0000gp/T/ipykernel_2391/4224608015.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SHar['rank'] = df_SHar['rank'].rank(ascending = True)


,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
3,1.0,False,False,ChiroxPrime,100,Slayer,c861372da792be0b22c45bf437ccd58437c52e9455e0bf...,4250334444,Chiroxun,26,NaN,SSF Harbinger,1,1,0,1,2
7,2.0,False,False,beobio_capitan,100,Inquisitor,65c58f9b186e1fbe9f927484f3f178cdb9ee0dc2427289...,4250334444,TishaVosemsot,40,beobio,SSF Harbinger,1,1,1,0,2
9,3.0,False,False,FutonBlewAway,100,Raider,46e56572ff6e403f45fac597240e13e9af225f06ca4e52...,4250334444,pleaGRC,40,NaN,SSF Harbinger,1,0,0,1,1
14,4.0,False,False,Kinther,100,Gladiator,aea0afdd0b84cec8418b4228345526304048cbdaf1d902...,4250334444,nehxzel,35,NaN,SSF Harbinger,1,1,0,1,2
19,5.0,False,False,Ironmaan_BTW,100,Necromancer,bace7ed50cbc71a3823c0728c8199d3af37e39042b7cbe...,4250334444,Pochtli,40,NaN,SSF Harbinger,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59756,14977.0,False,False,Matrone,73,Necromancer,0c2c5e2ba00dde255cb9f1fd0fe0e754b1c2eca6eb5d4b...,495267342,KingSlacker,13,NaN,SSF Harbinger,1,0,1,0,1
59760,14978.0,False,False,EQNivellen,73,Juggernaut,441515d2da7e6cdf9c6084b539cd07fd41ff244627dda5...,495239725,ColonelGhosn,22,NaN,SSF Harbinger,1,1,0,0,1
59764,14979.0,False,False,Sorryffs,73,Trickster,4708b0689eac845650f7040056d3e1c6a597a3121ec815...,495220075,AFKpirate,24,skoltrrs,SSF Harbinger,1,0,1,1,2
59769,14980.0,False,False,Reveil_Miner,73,Saboteur,43a9539f581a8af8f47769919c1d54a2e12e92d6a6e344...,495078061,infiym2,9,NaN,SSF Harbinger,1,0,1,1,2


In [26]:
# 정수 타입으로 바꾸어준다.

df_SHar['rank'] = df_SHar['rank'].astype('int64', copy = True)

/var/folders/qw/j72d9nwn5dq0vxtqtw2bbtnh0000gp/T/ipykernel_2391/2541560182.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SHar['rank'] = df_SHar['rank'].astype('int64', copy = True)


In [27]:
# checking

df_SHar

,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
3,1,False,False,ChiroxPrime,100,Slayer,c861372da792be0b22c45bf437ccd58437c52e9455e0bf...,4250334444,Chiroxun,26,NaN,SSF Harbinger,1,1,0,1,2
7,2,False,False,beobio_capitan,100,Inquisitor,65c58f9b186e1fbe9f927484f3f178cdb9ee0dc2427289...,4250334444,TishaVosemsot,40,beobio,SSF Harbinger,1,1,1,0,2
9,3,False,False,FutonBlewAway,100,Raider,46e56572ff6e403f45fac597240e13e9af225f06ca4e52...,4250334444,pleaGRC,40,NaN,SSF Harbinger,1,0,0,1,1
14,4,False,False,Kinther,100,Gladiator,aea0afdd0b84cec8418b4228345526304048cbdaf1d902...,4250334444,nehxzel,35,NaN,SSF Harbinger,1,1,0,1,2
19,5,False,False,Ironmaan_BTW,100,Necromancer,bace7ed50cbc71a3823c0728c8199d3af37e39042b7cbe...,4250334444,Pochtli,40,NaN,SSF Harbinger,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59756,14977,False,False,Matrone,73,Necromancer,0c2c5e2ba00dde255cb9f1fd0fe0e754b1c2eca6eb5d4b...,495267342,KingSlacker,13,NaN,SSF Harbinger,1,0,1,0,1
59760,14978,False,False,EQNivellen,73,Juggernaut,441515d2da7e6cdf9c6084b539cd07fd41ff244627dda5...,495239725,ColonelGhosn,22,NaN,SSF Harbinger,1,1,0,0,1
59764,14979,False,False,Sorryffs,73,Trickster,4708b0689eac845650f7040056d3e1c6a597a3121ec815...,495220075,AFKpirate,24,skoltrrs,SSF Harbinger,1,0,1,1,2
59769,14980,False,False,Reveil_Miner,73,Saboteur,43a9539f581a8af8f47769919c1d54a2e12e92d6a6e344...,495078061,infiym2,9,NaN,SSF Harbinger,1,0,1,1,2


##### df_SHarHC

In [28]:
# 기존 rank에 rank를 다시 매긴다. ascending = True : 작은 순대로 오름차순 1부터 순위 매김.

df_SHarHC['rank'] = df_SHarHC['rank'].rank(ascending = True)
df_SHarHC

/var/folders/qw/j72d9nwn5dq0vxtqtw2bbtnh0000gp/T/ipykernel_2391/2299381472.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SHarHC['rank'] = df_SHarHC['rank'].rank(ascending = True)


,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
1,1.0,False,False,RaizNeverFirstQT,100,Necromancer,8f3216db5ac9106c287a834731aafc83c387138f28fc45...,4250334444,RaizQT,22,raizqt,SSF Harbinger HC,1,0,1,0,1
6,2.0,False,False,Grom_WhyGodThreeCmWhy,100,Gladiator,5d8b80b2bdd30f3dba1036a6791187562758935e8d4723...,4250334444,rocklegend,22,gromick,SSF Harbinger HC,1,1,0,1,2
11,3.0,False,False,ShiruHBSSF,100,Slayer,b0cf3e837984f7f9c8e00d0b29d1e5670c88e045daf185...,4250334444,shirusen,23,NaN,SSF Harbinger HC,1,1,0,1,2
15,4.0,True,False,Dev_XD,100,Raider,dae26ebd7582552ba5330897c98409644cbbd35d3a879c...,4250334444,Develina,40,NaN,SSF Harbinger HC,1,0,0,1,1
17,5.0,False,False,ToTheHeartache,100,Inquisitor,ac9f63212fe2eb3be803fe3b5c4bab2643dcdf1be9a1b7...,4250334444,Realyn,38,nyylaer,SSF Harbinger HC,1,1,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59759,14968.0,True,False,Scrilinois,53,Trickster,8ee06894a715d9e5d0496d763d78d29e13203b333b58ba...,77936891,Notaufnahme,9,NaN,SSF Harbinger HC,1,0,1,1,2
59762,14969.0,True,False,Kuldian,53,Juggernaut,ae123ba62092ff22bc999c1687243d8836b06197b52f91...,77927580,kuldian_88,19,NaN,SSF Harbinger HC,1,1,0,0,1
59765,14970.0,True,False,pledge_of_hams,53,Berserker,56b1b9957550bc8f54727a682ca40b3a378ac84dfac8b1...,77925176,contemporaneously,24,NaN,SSF Harbinger HC,1,1,0,0,1
59768,14971.0,True,False,SSFHCRamboMustDieOften,53,Necromancer,0d4fc2dd7e295c8b9c6db4d6cc8e0036ea3789800fcf9c...,77914551,iskayfleyks,29,iskayfleyks,SSF Harbinger HC,1,0,1,0,1


In [29]:
# 정수 타입으로 바꾸어준다.

df_SHarHC['rank'] = df_SHarHC['rank'].astype('int64', copy = True)

/var/folders/qw/j72d9nwn5dq0vxtqtw2bbtnh0000gp/T/ipykernel_2391/2081354654.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SHarHC['rank'] = df_SHarHC['rank'].astype('int64', copy = True)


In [30]:
# checking

df_SHarHC

,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
1,1,False,False,RaizNeverFirstQT,100,Necromancer,8f3216db5ac9106c287a834731aafc83c387138f28fc45...,4250334444,RaizQT,22,raizqt,SSF Harbinger HC,1,0,1,0,1
6,2,False,False,Grom_WhyGodThreeCmWhy,100,Gladiator,5d8b80b2bdd30f3dba1036a6791187562758935e8d4723...,4250334444,rocklegend,22,gromick,SSF Harbinger HC,1,1,0,1,2
11,3,False,False,ShiruHBSSF,100,Slayer,b0cf3e837984f7f9c8e00d0b29d1e5670c88e045daf185...,4250334444,shirusen,23,NaN,SSF Harbinger HC,1,1,0,1,2
15,4,True,False,Dev_XD,100,Raider,dae26ebd7582552ba5330897c98409644cbbd35d3a879c...,4250334444,Develina,40,NaN,SSF Harbinger HC,1,0,0,1,1
17,5,False,False,ToTheHeartache,100,Inquisitor,ac9f63212fe2eb3be803fe3b5c4bab2643dcdf1be9a1b7...,4250334444,Realyn,38,nyylaer,SSF Harbinger HC,1,1,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59759,14968,True,False,Scrilinois,53,Trickster,8ee06894a715d9e5d0496d763d78d29e13203b333b58ba...,77936891,Notaufnahme,9,NaN,SSF Harbinger HC,1,0,1,1,2
59762,14969,True,False,Kuldian,53,Juggernaut,ae123ba62092ff22bc999c1687243d8836b06197b52f91...,77927580,kuldian_88,19,NaN,SSF Harbinger HC,1,1,0,0,1
59765,14970,True,False,pledge_of_hams,53,Berserker,56b1b9957550bc8f54727a682ca40b3a378ac84dfac8b1...,77925176,contemporaneously,24,NaN,SSF Harbinger HC,1,1,0,0,1
59768,14971,True,False,SSFHCRamboMustDieOften,53,Necromancer,0d4fc2dd7e295c8b9c6db4d6cc8e0036ea3789800fcf9c...,77914551,iskayfleyks,29,iskayfleyks,SSF Harbinger HC,1,0,1,0,1


# data concat and export

이 랭킹을 앞으로도 쓸 것이기 때문에 일단 수정한 채로 export 할 것입니다...


In [31]:
df_all = pd.concat((df_Har, df_HarHC, df_SHar, df_SHarHC), axis = 0)

In [32]:
df_all.sort_index(ascending = True, inplace = True)
df_all

,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
0,1,False,False,Tzn_NecroIsFineNow,100,Necromancer,3dcddd59f5088893f734f39686350990dae168cc4f4b00...,4250334444,TheTzn,40,thetzn,Harbinger,1,0,1,0,1
1,1,False,False,RaizNeverFirstQT,100,Necromancer,8f3216db5ac9106c287a834731aafc83c387138f28fc45...,4250334444,RaizQT,22,raizqt,SSF Harbinger HC,1,0,1,0,1
2,1,False,False,GucciStreamerAdvantage,100,Necromancer,c6ec2dae3855c551e0597c06ef2da06fbb5512487ded7f...,4250334444,GucciPradas,35,guccipradas,Hardcore Harbinger,1,0,1,0,1
3,1,False,False,ChiroxPrime,100,Slayer,c861372da792be0b22c45bf437ccd58437c52e9455e0bf...,4250334444,Chiroxun,26,NaN,SSF Harbinger,1,1,0,1,2
4,2,False,False,Cool_NecroIsFineNow,100,Deadeye,24ae924ceed7989ef3d3d6772612832bb467a6094352c7...,4250334444,cooltail,22,NaN,Harbinger,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59771,14904,True,False,ПроклятьеРекласта,89,Necromancer,d33b4f6e08c10e365765f9a36a8f36d561fd1d86f10ee1...,1786695653,Romanchik002,18,romanchik002,Hardcore Harbinger,1,0,1,0,1
59772,14918,False,False,IshibashiSummoner,94,Necromancer,5764cfa387e0a87a4bebc1a3c5017e92de8bbb06445c47...,2720770853,Ishibashisan,33,NaN,Harbinger,1,0,1,0,1
59773,14981,False,False,BLively,73,Slayer,9ac75ab75a47cee8a9dfb0a31912df8909720a8b20f9f1...,495034029,ruby01,19,NaN,SSF Harbinger,1,1,0,1,2
59774,14905,False,False,vawddvaw,89,Gladiator,cf02dfc0c90b2df9c7ac76bbedd91e93c2a8a2ca629c15...,1786638972,Lecarn,24,lecarn,Hardcore Harbinger,1,1,0,1,2


In [33]:
df_all.to_csv('poe_stats_2revisedrank.csv', index = False)

In [34]:
# Let's check the file

thefile = pd.read_csv('poe_stats_2revisedrank.csv')
thefile

,rank,dead,online,name,level,class,id,experience,account,challenges,twitch,ladder,special_class,stat_Str,stat_Int,stat_Dex,stats_num
0,1,False,False,Tzn_NecroIsFineNow,100,Necromancer,3dcddd59f5088893f734f39686350990dae168cc4f4b00...,4250334444,TheTzn,40,thetzn,Harbinger,1,0,1,0,1
1,1,False,False,RaizNeverFirstQT,100,Necromancer,8f3216db5ac9106c287a834731aafc83c387138f28fc45...,4250334444,RaizQT,22,raizqt,SSF Harbinger HC,1,0,1,0,1
2,1,False,False,GucciStreamerAdvantage,100,Necromancer,c6ec2dae3855c551e0597c06ef2da06fbb5512487ded7f...,4250334444,GucciPradas,35,guccipradas,Hardcore Harbinger,1,0,1,0,1
3,1,False,False,ChiroxPrime,100,Slayer,c861372da792be0b22c45bf437ccd58437c52e9455e0bf...,4250334444,Chiroxun,26,NaN,SSF Harbinger,1,1,0,1,2
4,2,False,False,Cool_NecroIsFineNow,100,Deadeye,24ae924ceed7989ef3d3d6772612832bb467a6094352c7...,4250334444,cooltail,22,NaN,Harbinger,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59771,14904,True,False,ПроклятьеРекласта,89,Necromancer,d33b4f6e08c10e365765f9a36a8f36d561fd1d86f10ee1...,1786695653,Romanchik002,18,romanchik002,Hardcore Harbinger,1,0,1,0,1
59772,14918,False,False,IshibashiSummoner,94,Necromancer,5764cfa387e0a87a4bebc1a3c5017e92de8bbb06445c47...,2720770853,Ishibashisan,33,NaN,Harbinger,1,0,1,0,1
59773,14981,False,False,BLively,73,Slayer,9ac75ab75a47cee8a9dfb0a31912df8909720a8b20f9f1...,495034029,ruby01,19,NaN,SSF Harbinger,1,1,0,1,2
59774,14905,False,False,vawddvaw,89,Gladiator,cf02dfc0c90b2df9c7ac76bbedd91e93c2a8a2ca629c15...,1786638972,Lecarn,24,lecarn,Hardcore Harbinger,1,1,0,1,2
